In [1]:
import json

# Specify the file path
file_path = "mistral-7b-5-RT-5_epochs_test.json"
def preprocess_text(text):
  return text
# Read the file
with open(file_path, "r", encoding='utf-8') as f:
    test_data = json.load(f)
    answers = list(map(lambda x: x['gold_query'], test_data))
    question = list(map(lambda x: preprocess_text(x['question']), test_data))
    test_y = list(map(lambda x: preprocess_text(x['predicted_query']), test_data))
    #ans = list(map(lambda x: preprocess_text(x['answers']), test_data))
    new_answers = list(map(lambda x: preprocess_text(x['new_answers']), test_data))
    #formulated_query = list(map(lambda x: preprocess_text(x['formulated_prompt']), test_data))
    ids = list(map(lambda x: x['id'], test_data))
# Print the data
print(test_data)


[{'id': '99', 'question': 'What is the time zone of Salt Lake City?', 'gold_query': 'PREFIX res: <http://dbpedia.org/resource/> PREFIX dbp: <http://dbpedia.org/property/> SELECT DISTINCT ?uri WHERE { res:Salt_Lake_City <http://dbpedia.org/ontology/timeZone> ?uri }', 'predicted_query': "[INST] Context information is below.\n---------------------\n<http://dbpedia.org/resource/Salt_Lake_City> <http://dbpedia.org/property/timezone> <http://dbpedia.org/resource/Mountain_Time_Zone> <sep> <http://dbpedia.org/resource/Salt_Lake_City> <http://dbpedia.org/ontology/timeZone> <http://dbpedia.org/resource/Mountain_Time_Zone> <sep> <http://dbpedia.org/resource/Salt_Lake_City> <http://dbpedia.org/ontology/type> <http://dbpedia.org/resource/List_of_capitals_in_the_United_States> <sep> <http://dbpedia.org/resource/Nick_Gage__Nick_Gage__1> <http://dbpedia.org/ontology/billed> <http://dbpedia.org/resource/Salt_Lake_City> <sep> <http://dbpedia.org/resource/Utah_Starzz__Utah_Starzz__1> <http://dbpedia.org/

In [2]:
empty_count = sum([1 for a in new_answers if not a])
print(f"Number of empty ans: {empty_count}")

Number of empty ans: 33


In [3]:
results = []
queries = list(zip(answers, test_y))

In [4]:
from tqdm import trange
for i in trange(len(queries)):
    gold, test_case= queries[i]
    error_count = 0
    #print(test_case)
    try:
        split_text = test_case.split("The correct SPARQL query to answer the question is")

        extracted_text = split_text[1].strip()
        #print(extracted_text)
        results.append({'id': ids[i], 'question': question[i], 'gold_query': gold, 'predicted_query': extracted_text, 'new_answers':new_answers[i]})
    except:
        try:
            print("Error in test case: ", ids[i], "we will try to split at \'question is\' instead")
            split_text = test_case.split("question is")
            extracted_text = split_text[1].strip()
            print(extracted_text)
            results.append({'id': ids[i], 'question': question[i], 'gold_query': gold, 'predicted_query': extracted_text, 'new_answers':new_answers[i]})
        except:
            try:
                print("there was still an error in test case: ", ids[i], 'trying to split at \'questions is\'')
                split_text = test_case.split("questions is")
                extracted_text = split_text[1].strip()
                print(extracted_text)
                results.append({'id': ids[i], 'question': question[i], 'gold_query': gold, 'predicted_query': extracted_text, 'new_answers':new_answers[i]})
            except:
                print("there was still an error in test case: ", ids[i], 'trying to split at \'query is\'')
                split_text = test_case.split("query is")
                extracted_text = split_text[1].strip()
                print(extracted_text)
                results.append({'id': ids[i], 'question': question[i], 'gold_query': gold, 'predicted_query': extracted_text, 'new_answers':new_answers[i]})


                # print("Error in test case: ", ids[i])
                # print(test_case)
        
            
    #split_text = test_case.split("The correct SPARQL query to answer the questions is")
    #results.append({'id': ids[i], 'question': question[i], 'gold_query': gold, 'predicted_query': extracted_text, 'answers':ans[i], 'new_answers': new_answers[i], 'formulated_query': formulated_query[i] })
    #results.append({'id': ids[i], 'question': question[i], 'gold_query': gold, 'predicted_query': extracted_text, 'answers':ans[i], 'formulated_query': formulated_query[i] })



100%|██████████| 150/150 [00:00<00:00, 128370.86it/s]


In [5]:
with open('mistral-7b-5-RT-5_epochs_test_parsed.json', 'w', encoding='utf-8') as outfile:
    json.dump(results, outfile, indent=4)

In [ ]:
import json
file_path = "LCQUAD1_train_final.json"
def preprocess_text(text):
  return text
# Read the file
with open(file_path, "r") as f:
    test_data = json.load(f)
    query = list(map(lambda x: x['query'], test_data))
    answers = list(map(lambda x: x['answers'], test_data))
    question = list(map(lambda x: x['question'], test_data))
    triples = list(map(lambda x: preprocess_text(x['triples']), test_data))
    new_triples = list(map(lambda x: preprocess_text(x['new_triples']), test_data))
    ids = list(map(lambda x: x['id'], test_data))
# Print the data
print(test_data)


In [ ]:
import re
def transform_string(s):
    # Splitting the string into two parts based on ':'
    parts = s.split('>:')
    
    # If split is successful and length is 2, modify the string
    if len(parts) == 2:
        return parts[0] + parts[1].replace(">:", "") + '>'
    else:
        return s
def surround_https_strings(text):
    if text.startswith('http:'):
        return '<' + text + '>'
    else:
        return text
def transform_json_data(data):
    for sublist in data:
        for i, item in enumerate(sublist):
            if "http://" in item:
                sublist[i] = transform_string(item)
    return data
results = []
ziptriples = list(zip(triples, new_triples))

In [ ]:
# Example JSON data
# json_data = [
#     ["?sub", "<http://dbpedia.org/ontology/>:goldMedalist", "<http://dbpedia.org/resource/>:Michael_Phelps"]
# ]
from tqdm import trange
for i in trange(len(ziptriples)):
    trips, new_trips= ziptriples[i]
    #print(test_case)
    for j in range(len(trips)):
        trips[j]=transform_json_data(trips[j])
    for k in range(len(new_trips)):
        new_trips[k]=transform_json_data(new_trips[k])
        for j in range(len(new_trips[k])):
            #print(new_trips[k][j])
            for l in range(len(new_trips[k][j])):
                new_trips[k][j][l]=surround_https_strings(new_trips[k][j][l])
    results.append({'id': ids[i],'query': query[i], 'question': question[i], 'answers': answers[i],'triples': trips, 'new_triples': new_trips})

# # Transform the data
# transformed_data = transform_json_data(json_data)

# # Convert the transformed data back to JSON
# json_output = json.dumps(transformed_data, indent=4)
# print(json_output)

In [ ]:
with open('LCQUAD1_train_final.json', 'w') as outfile:
    json.dump(results, outfile, indent=4)

In [ ]:
import json

# Specify the file path
file_path = "LCQUAD1_train_final.json"
def preprocess_text(text):
  return text
# Read the file
with open(file_path, "r") as f:
    test_data = json.load(f)
    query = list(map(lambda x: x['query'], test_data))
    question = list(map(lambda x: preprocess_text(x['question']), test_data))
    answer = list(map(lambda x: preprocess_text(x['answers']), test_data))
    ids = list(map(lambda x: x['id'], test_data))
    triples = list(map(lambda x: preprocess_text(x['triples']), test_data))
    new_triples = list(map(lambda x: preprocess_text(x['new_triples']), test_data))
# Print the data
print(test_data)


In [ ]:
def clean_triples(data):
    # Function to check if a triple is empty
    def is_empty_triple(triple):
        return len(triple) == 1 and triple[0] == ""

    # Clean 'triples'
    cleaned_triples = []
    for group in data['triples']:
        cleaned_group = [triple for triple in group if not is_empty_triple(triple)]
        if cleaned_group:  # Only add non-empty groups
            cleaned_triples.append(cleaned_group)
    data['triples'] = cleaned_triples

    # Clean 'new_triples'
    cleaned_new_triples = []
    for group in data['new_triples']:
        cleaned_group = [triple for triple in group if not is_empty_triple(triple)]
        if cleaned_group:  # Only add non-empty groups
            cleaned_new_triples.append(cleaned_group)
    data['new_triples'] = cleaned_new_triples

    return data



In [ ]:
results = []

In [ ]:
from tqdm import trange
for i in trange(len(test_data)):
    processed_data = clean_triples(test_data[i])
    #print(processed_data)
    print(processed_data['id'])
    results.append({'id': processed_data['id'],'query': processed_data['query'], 'question': processed_data['question'], 'answers': processed_data['answers'],'triples': processed_data['triples'], 'new_triples': processed_data['new_triples']})


In [ ]:
with open('LCQUAD1_train_final.json', 'w') as outfile:
    json.dump(results, outfile, indent=4)

remove dbr: and add >

In [ ]:
import json

# Specify the file path
file_path = "LCQUAD1_train_final_with_bm25_triples_with_noisy_triples.json"
def preprocess_text(text):
  return text
# Read the file
with open(file_path, "r", encoding='utf-8') as f:
    test_data = json.load(f)
    question = list(map(lambda x: preprocess_text(x['question']), test_data))
    ans = list(map(lambda x: preprocess_text(x['answers']), test_data))
    new_triples = list(map(lambda x: preprocess_text(x['new_triples']), test_data))
    bm25_triples = list(map(lambda x: preprocess_text(x['bm25_triples']), test_data))
    noisy_triples = list(map(lambda x: preprocess_text(x['noisy_triples']), test_data))
    ids = list(map(lambda x: x['id'], test_data))
# Print the data
#print(test_data)
with open('LCQUAD1_train_final.json', "r", encoding='utf-8') as f2:
    test_data2 = json.load(f2)
    query = list(map(lambda x: x['query'], test_data2))


In [ ]:
results = []

In [ ]:
for i in range(len(question)):
    for j in range(len(noisy_triples[i])):
        for k in range(len(noisy_triples[i][j])):
            #print(noisy_triples[i][j][k])
            if "dbr:" in noisy_triples[i][j][k]:
                noisy_triples[i][j][k] = noisy_triples[i][j][k].replace("dbr:", "<http://dbpedia.org/resource/") + ">"
            #print(noisy_triples[i][j][k])
    results.append({'id': ids[i], 'question': question[i], 'query':query[i], 'answers': ans[i], 'new_triples': new_triples[i], 'bm25_triples': bm25_triples[i], 'noisy_triples': [noisy_triples[i]]})

In [ ]:
print(results[2])

In [ ]:
with open('LCQUAD1_train_final_with_bm25_triples_with_noisy_triples.json', 'w', encoding='utf-8') as outfile:
    json.dump(results, outfile, indent=4)